In [1]:
from IPython.utils import io
import csv
with io.capture_output() as captured:  
   %run ../../2-FeatureExtraction/TF-IDF/TF-IDF.ipynb import ngram_tfidf_feature, y1, y2, dev_ngram_feature, Y1, Y2, vocabulary, Preprocessing, Features

In [2]:
from sklearn import svm
from sklearn.metrics import f1_score, classification_report

#### SVM Training

In [3]:
kernel = 'poly'
WeightedLoss = True
model = svm.SVC(kernel=kernel, class_weight='balanced' if WeightedLoss else None )
model = model.fit(ngram_tfidf_feature, y1)
ModelInfo = {
   "Model": "SVM-1",
   "Kernel":kernel,
   "Weighted Loss": WeightedLoss
}

#### SVM Prediction

In [4]:
outputs = model.predict(dev_ngram_feature)
Report = classification_report(Y1, outputs, output_dict=True)
print(classification_report(Y1, outputs))
F1 = f1_score(Y1, outputs, average='macro')
print('F1 score: ', F1)


              precision    recall  f1-score   support

           0       0.48      0.37      0.42       126
           1       0.86      0.93      0.90       804
           2       0.38      0.17      0.24        70

    accuracy                           0.81      1000
   macro avg       0.57      0.49      0.52      1000
weighted avg       0.78      0.81      0.79      1000

F1 score:  0.5178450664115455


#### Log Model Info

In [5]:
with open('runs.csv', 'a') as f:  
      run_info = {**Preprocessing, **Features, **ModelInfo, "acc":Report["accuracy"],"BF1": Report["macro avg"]["f1-score"], "WF1": Report["weighted avg"]["f1-score"] }
      w = csv.DictWriter(f, run_info.keys())
      w.writeheader()
      w.writerow(run_info)